# Conflict Resolution Strategies

## Overview

Detect conflicts in knowledge graphs, apply multiple resolution strategies, track sources, and maintain audit trails.

## Workflow: Detect Conflicts → Multiple Resolution Strategies → Track Sources → Audit


In [ ]:
from semantica.kg import GraphBuilder
from semantica.kg_qa import ConsistencyChecker
from datetime import datetime
import json


## Step 1: Create Knowledge Graph with Conflicting Data


In [ ]:
builder = GraphBuilder()

entities = [
    {
        "id": "e1",
        "type": "Person",
        "name": "John Doe",
        "properties": {"age": 30, "location": "New York"},
        "source": "source1",
        "timestamp": datetime(2023, 1, 1)
    },
    {
        "id": "e1",
        "type": "Person",
        "name": "John Doe",
        "properties": {"age": 32, "location": "Boston"},
        "source": "source2",
        "timestamp": datetime(2023, 6, 1)
    },
    {
        "id": "e2",
        "type": "Organization",
        "name": "Tech Corp",
        "properties": {"founded": 2010, "employees": 100},
        "source": "source1",
        "timestamp": datetime(2023, 1, 1)
    },
    {
        "id": "e2",
        "type": "Organization",
        "name": "Tech Corp",
        "properties": {"founded": 2012, "employees": 150},
        "source": "source2",
        "timestamp": datetime(2023, 3, 1)
    },
]

relationships = [
    {"source": "e1", "target": "e2", "type": "works_for", "source": "source1"},
    {"source": "e1", "target": "e2", "type": "founder_of", "source": "source2"},
]

knowledge_graph = builder.build(entities, relationships)


## Step 2: Detect Conflicts


In [ ]:
consistency_checker = ConsistencyChecker()
conflicts = consistency_checker.check_conflicts(knowledge_graph)

for i, conflict in enumerate(conflicts, 1):
    print(f"Conflict {i}:")
    print(f"  Entity/Relationship: {conflict.get('entity_id', conflict.get('relationship_id'))}")
    print(f"  Type: {conflict.get('type')}")
    print(f"  Conflicting values: {conflict.get('values')}")
    print(f"  Sources: {conflict.get('sources')}")


## Step 3: Multiple Resolution Strategies


In [ ]:
class ConflictResolver:
    def __init__(self):
        self.audit_trail = []
    
    def resolve(self, conflicts, strategy="most_recent"):
        resolved = []
        
        for conflict in conflicts:
            if strategy == "most_recent":
                values = conflict.get('values', [])
                timestamps = conflict.get('timestamps', [])
                if timestamps:
                    most_recent_idx = timestamps.index(max(timestamps))
                    resolved_value = values[most_recent_idx]
                else:
                    resolved_value = values[-1] if values else None
                    
            elif strategy == "authoritative":
                sources = conflict.get('sources', [])
                authoritative_sources = ["source1", "official_db", "verified"]
                resolved_value = None
                for auth_source in authoritative_sources:
                    if auth_source in sources:
                        idx = sources.index(auth_source)
                        resolved_value = conflict.get('values', [])[idx]
                        break
                if resolved_value is None:
                    resolved_value = conflict.get('values', [])[0] if conflict.get('values') else None
                    
            elif strategy == "merge":
                values = conflict.get('values', [])
                if isinstance(values[0], dict):
                    merged = {}
                    for val in values:
                        merged.update(val)
                    resolved_value = merged
                elif isinstance(values[0], (int, float)):
                    resolved_value = sum(values) / len(values)
                else:
                    resolved_value = ", ".join(set(str(v) for v in values))
            else:
                resolved_value = conflict.get('values', [])[0] if conflict.get('values') else None
            
            resolved.append({
                'conflict_id': conflict.get('entity_id', conflict.get('relationship_id')),
                'resolved_value': resolved_value,
                'strategy': strategy,
                'timestamp': datetime.now()
            })
            
            self.audit_trail.append({
                'conflict': conflict,
                'resolution': resolved[-1],
                'resolved_at': datetime.now()
            })
        
        return resolved

resolver = ConflictResolver()

resolved_1 = resolver.resolve(conflicts, strategy="most_recent")
print("Strategy 1: Most Recent Wins")
for r in resolved_1:
    print(f"  Resolved: {r['conflict_id']} = {r['resolved_value']}")

resolver2 = ConflictResolver()
resolved_2 = resolver2.resolve(conflicts, strategy="authoritative")
print("\nStrategy 2: Most Authoritative Source Wins")
for r in resolved_2:
    print(f"  Resolved: {r['conflict_id']} = {r['resolved_value']}")

resolver3 = ConflictResolver()
resolved_3 = resolver3.resolve(conflicts, strategy="merge")
print("\nStrategy 3: Merge Conflicting Information")
for r in resolved_3:
    print(f"  Resolved: {r['conflict_id']} = {r['resolved_value']}")


## Step 4: Track Sources


In [ ]:
class SourceTracker:
    def __init__(self):
        self.source_map = {}
    
    def track_sources(self, conflicts):
        for conflict in conflicts:
            conflict_id = conflict.get('entity_id', conflict.get('relationship_id'))
            sources = conflict.get('sources', [])
            timestamps = conflict.get('timestamps', [])
            
            self.source_map[conflict_id] = {
                'sources': sources,
                'timestamps': timestamps,
                'values': conflict.get('values', [])
            }
    
    def get_sources(self, conflict):
        conflict_id = conflict.get('entity_id', conflict.get('relationship_id'))
        return self.source_map.get(conflict_id, {})

tracker = SourceTracker()
tracker.track_sources(conflicts)

for conflict in conflicts:
    sources = tracker.get_sources(conflict)
    conflict_id = conflict.get('entity_id', conflict.get('relationship_id'))
    print(f"Conflict: {conflict_id}")
    print(f"  Sources: {sources.get('sources', [])}")
    print(f"  Timestamps: {sources.get('timestamps', [])}")
    print(f"  Values: {sources.get('values', [])}")


## Step 5: Audit Trail


In [ ]:
audit_log = resolver.get_audit_trail() if hasattr(resolver, 'get_audit_trail') else resolver.audit_trail

for i, entry in enumerate(audit_log, 1):
    print(f"Entry {i}:")
    print(f"  Conflict ID: {entry['conflict'].get('entity_id', entry['conflict'].get('relationship_id'))}")
    print(f"  Resolution Strategy: {entry['resolution']['strategy']}")
    print(f"  Resolved Value: {entry['resolution']['resolved_value']}")
    print(f"  Resolved At: {entry['resolved_at']}")

audit_export = []
for entry in audit_log:
    audit_export.append({
        'conflict_id': entry['conflict'].get('entity_id', entry['conflict'].get('relationship_id')),
        'conflict_type': entry['conflict'].get('type'),
        'original_values': entry['conflict'].get('values'),
        'sources': entry['conflict'].get('sources'),
        'resolution_strategy': entry['resolution']['strategy'],
        'resolved_value': str(entry['resolution']['resolved_value']),
        'resolved_at': entry['resolved_at'].isoformat()
    })


## Summary

Conflict resolution workflow:
- Conflict Detection
- Multiple Resolution Strategies (Most Recent, Authoritative, Merge)
- Source Tracking
- Complete Audit Trail


In [ ]:
print(f"Detected {len(conflicts)} conflicts")
print(f"Applied 3 resolution strategies")
print(f"Maintained audit trail with {len(audit_log)} entries")
